In [1]:
# EJERCICIO 1 - APIs

# para hacer solicitudes HTTP
import requests

# para manipulación de datos
import pandas as pd

In [10]:
# cada una pone la kpi que haya generado

api_key = "12b5dc93-7799-439e-ac66-3b0eda416a3f"

# 12b5dc93-7799-439e-ac66-3b0eda416a3f - este es el apikey que sacó Nerea

# Primero hacemos 

jugadores = {"nombres":[], "apellidos":[], "equipo":[]}

# Estos son los parámetros de la solicitud. Vamos a pedir los 200 jugadores, para que nos muestre 100 por página:
parametros = {'page': 200,'per_page': 100}

res_jugadores = requests.get(f"http://api.balldontlie.io/v1/players", params = parametros, headers = {'Authorization': api_key} ) 

# llamada de los jugadores:
# el json() nos sirve para estructurar los datos que intercambian el servidor y la llamada

res_jugadores = res_jugadores.json()


In [11]:
# Explicacion del FOR: por cada página en el rango primera página a la tercera, metemos otro for dentro, para recorrer todos los jugadores:

for page in range(1, 3):
    for jugador in res_jugadores["data"]:
        jugadores["nombres"].append(jugador["first_name"])
        jugadores["apellidos"].append(jugador["last_name"])
        jugadores["equipo"].append(jugador["team"])

# estructuracion de datos

dataframe_jugadores = pd.DataFrame(jugadores)
dataframe_jugadores

,nombres,apellidos,equipo
0,Alex,Abrines,"{'id': 21, 'conference': 'West', 'division': '..."
1,Jaylen,Adams,"{'id': 1, 'conference': 'East', 'division': 'S..."
2,Steven,Adams,"{'id': 11, 'conference': 'West', 'division': '..."
3,Bam,Adebayo,"{'id': 16, 'conference': 'East', 'division': '..."
4,DeVaughn,Akoon-Purcell,"{'id': 8, 'conference': 'West', 'division': 'N..."
...,...,...,...
195,Joe,Chealey,"{'id': 4, 'conference': 'East', 'division': 'S..."
196,Marquese,Chriss,"{'id': 7, 'conference': 'West', 'division': 'S..."
197,Gary,Clark,"{'id': 19, 'conference': 'West', 'division': '..."
198,Ian,Clark,"{'id': 10, 'conference': 'West', 'division': '..."


In [29]:
partidos_request = requests.get(f"https://api.balldontlie.io/v1/games?per_page=100?page={pagina}" )
partidos_request.status_code

401

In [23]:
# Extraer la información de 200 partidos:

partidos = {"equipo_local":[], "equipo_visitante":[],"puntos_local":[], "puntos_visitante": [],"temporada":[]}

# Hacemos el request dentro del for porque se hace un request por cada página.

for pagina in range (1, 3):

    try:
        partidos_request = requests.get(f"https://api.balldontlie.io/api/v1/games?per_page=100?page={pagina}" ) 
        partidos_request = partidos_request.json()

        for partido in partidos_request["data"]:
            partidos["equipo_local"].append(partido["home_team"]["full_name"])
            partidos["equipo_visitante"].append(partido["visitor_team"]["full_name"])
            partidos["puntos_local"].append(partido["home_team_score"])
            partidos["puntos_visitante"].append(partido["visitor_team_score"])   
            partidos["temporada"].append(partido["season"]) 

    except requests.exceptions.RequestException as e:
        print(f"Error al realizar la solicitud en la página {pagina}: {e}")
    
    except ValueError as e:

        print(f"Error al convertir la respuesta a JSON en la página {pagina}: {e}")


partidos_dataframe = pd.DataFrame(partidos)                          
partidos_dataframe


Error al realizar la solicitud en la página 1: Expecting value: line 1 column 1 (char 0)
Error al realizar la solicitud en la página 2: Expecting value: line 1 column 1 (char 0)


,equipo_local,equipo_visitante,puntos_local,puntos_visitante,temporada


In [27]:
# ESTE ES EL DE LOS EJERCICIOS RESUELTOS:

partidos = {"equipo_local": [], "equipo_visitante": [], "puntos_local": [], "puntos_visitante": [], "temporada": []}

for pagina in range(1, 3):
        
    res_partidos = requests.get(f"https://www.balldontlie.io/api/v1/games?per_page=100?page={pagina}")
    res_partidos = res_partidos.json()

    
    for partido in res_partidos["data"]:
        partidos["equipo_local"].append(partido["home_team"]["full_name"])
        partidos["equipo_visitante"].append(partido["visitor_team"]["full_name"])
        partidos["puntos_local"].append(partido["home_team_score"])  
        partidos["puntos_visitante"].append(partido["visitor_team_score"])   
        partidos["temporada"].append(partido["season"]) 

df_partidos = pd.DataFrame(partidos)
df_partidos

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

## PARTE 2 WEB SCRAPPING con BEAUTIFUL SOUP



In [28]:
# antes de empezar importamos las librerías que vamos a usar. 
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [47]:
# FUNCION 

def hacer_medallas(url):

    request_medalla = requests.get(url) # primero hacemos la request.

    print(request_medalla.status_code) # miramos el status,

    soup = BeautifulSoup(request_medalla.content, "html.parser") # metemos el contenido en la sopita

    tablas = soup.find_all("table") # esto nos busca las tablas en el contenido

    medallero = tablas[0] # nos quedamos con la primera tabla, la guardamos en esta variable

    medallero_diccionario = {"posicion":[], "pais":[], "medallas_oro":[], "medallas_plata":[],"medallas_bronce":[], "total":[]}

    filas = medallero.findAll("tr")[1:] # nos busca todos los tr desde la segunda posición

    for fila in filas:
        celdas = fila.findAll("td")
        medallero_diccionario["posicion"].append(celdas[0].text)
        medallero_diccionario["pais"].append(celdas[1].text)
        medallero_diccionario["medallas_oro"].append(celdas[2].text)
        medallero_diccionario["medallas_plata"].append(celdas[3].text)
        medallero_diccionario["medallas_bronce"].append(celdas[4].text)
        medallero_diccionario["total"].append(celdas[5].text.strip())

    return pd.DataFrame(medallero_diccionario)


In [48]:
llamar_medallas = hacer_medallas("https://es.wikipedia.org/wiki/Anexo:Medallero_de_los_Juegos_Ol%C3%ADmpicos_de_Tokio_2020")
llamar_medallas


200


,posicion,pais,medallas_oro,medallas_plata,medallas_bronce,total
0,1,Estados Unidos (USA),39,41,33,113
1,2,China (CHN),38,32,19,89
2,3,Japón (JPN),27,14,17,58
3,4,Reino Unido (GBR),22,20,22,64
4,5,ROC (ROC),20,28,23,71
...,...,...,...,...,...,...
89,86,Granada (GRN),0,0,1,1
90,86,Kuwait (KUW),0,0,1,1
91,86,Moldavia (MDA),0,0,1,1
92,86,Siria (SYR),0,0,1,1
